In [4]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### gpt4-vision API를 활용하여 Image 인식 기반 답변 생성하기

##### 1. GPT vision api 사용해보기

In [1]:
from openai import OpenAI

client = OpenAI()

image_url = "https://imgnews.pstatic.net/image/092/2024/07/23/0002339294_001_20240723192911099.jpg?type=w647"
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "해당 사진에 대한 설명을 써줘"},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": image_url
                    },
                }
            ]
        }
    ]
)


print(chat_completion.choices[0].message.content)


사진에는 유명한 건축물인 그랑팔레(Grand Palais)가 나타나 있습니다. 이 건물은 아름다운 유리 지붕과 웅장한 고전적 디자인으로 잘 알려져 있으며, 여러 행사와 전시회가 개최되는 장소입니다. 사진의 왼쪽에는 에펠탑이 보이며, 맑은 하늘 아래 많은 사람들이 모여있는 모습이 인상적입니다. 또한, 건물의 외벽에는 올림픽 관련 배너가 걸려 있어 특정 이벤트와 관련된 상황임을 나타냅니다. 전체적으로 활기차고 다채로운 분위기를 자아내는 장면입니다.


##### 2. 이미지 기반 답변 생성 챗봇 만들기

In [12]:
from openai import OpenAI

client = OpenAI()

image_url = "https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/img/plan-and-execute.png"
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "위 이미지를 바탕으로 전체 흐름도에 대해 설명해줘"},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": image_url
                    },
                }
            ]
        }
    ]
)

print(chat_completion.choices[0].message.content)


이 흐름도는 사용자 요청에 따라 작업을 계획하고 실행하는 과정을 나타냅니다. 전체 프로세스는 다음과 같습니다:

1. **사용자 요청**: 사용자가 어떤 요구사항이나 질문을 제시합니다.
   
2. **작업 생성**: 사용자의 요청을 바탕으로 필요한 작업 목록이 생성됩니다.

3. **작업 실행**: 생성된 작업 목록에서 각 작업이 실행됩니다.

4. **상태 업데이트**: 각 작업이 완료된 후 결과를 바탕으로 현재 상태를 обнов합니다.

5. **재계획** (옵션):
   - 5a: 사용자가 요청한 내용에 따라 응답합니다.
   - 5b: 필요에 따라 추가 작업을 재계획합니다.

이 흐름도는 사용자와의 상호작용에 따라 지능적인 수행을 목적으로 하는 시스템의 구조를 명확하게 보여줍니다.


In [2]:
def image_based_answer(image_url, query):
  chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
              "role": "system",
              "content": "너는 흐름도를 바탕으로 답변해 주는 인공지능 챗봇이야. 사용자의 질문에 대해 친절하게 답변해줘"
        },
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": image_url
                    },
                }
            ]
        }
    ]
  )
  return chat_completion.choices[0].message.content

query_list = ["흐름도에서 업무생성에 대해 설명해주세요", "replan은 언제 발생하나요?"]
image_url = "https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/img/plan-and-execute.png"

for q in query_list:
  print(f"Q: {q}")
  print(f"A: {image_based_answer(image_url,  q)}")
  print("\n===============================\n")

Q: 흐름도에서 업무생성에 대해 설명해주세요
A: 흐름도에서 "업무 생성" 단계는 사용자 요청을 바탕으로 진행됩니다. 이 과정은 다음과 같은 단계로 설명할 수 있습니다:

1. **사용자 요청 (User Request)**: 사용자가 특정 작업이나 요구 사항을 제시합니다.
   
2. **업무 목록 생성 (Generate Tasks)**: 요청을 분석하여 필요한 작업들의 목록을 생성합니다. 이 목록은 해결해야 할 제반 업무를 포함합니다.

3. **업무 실행 (Exec Tasks)**: 생성된 업무 목록 중 한 가지를 선택하여 실행합니다. 이 단계에서는 실행할 작업이 단일 업무인 경우, 해당 업무를 처리하는 단일 작업 에이전트가 활용됩니다.

4. **상태 업데이트 (Update state with task results)**: 각 작업이 완료된 후 그 결과를 기반으로 시스템의 상태를 업데이트합니다.

5. **재계획 (Replan)**: 만약 추가적인 작업이 필요하다면, 다시 계획을 세워 새로운 작업을 생성하거나 기존 업무 목록을 수정합니다.

6. **사용자 응답 (Respond to user)**: 작업이 완료된 후, 사용자의 요청에 대한 결과를 다시 전달합니다.

이러한 흐름은 효율적인 작업 처리를 위해 반복적으로 수행될 수 있습니다.


Q: replan은 언제 발생하나요?
A: Replan은 'Task List' 단계에서 실행되는 작업이 예상대로 진행되지 않거나 실패할 경우 발생합니다. 이 단계에서 새로운 작업을 생성하거나 수정하여 원래의 계획을 조정합니다. 구체적으로, 'Replan'은 5b에서 이루어지며, 작업을 재구성하여 사용자 요청에 효과적으로 대응하기 위해 필요한 경우 발생합니다.




### 3. Multimodal Chain-of-Thought Prompting 을 사용하여 이미지에 대한 답변 생성하기

In [14]:
def generate_image_details(image_url):
  chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "해당 이미지에 대해 상세하게 설명해줘"},
                {
                    "type": "image_url",
                    "image_url": {
                      "url": image_url
                    },
                }
            ]
        }
    ]
  )

  image_details = chat_completion.choices[0].message.content
  return image_details

def text_based_answer(details, query):
  system_prompt = f"""
  너는 흐름도에 대한 설명을 바탕으로 답변해 주는 인공지능 챗봇이야. 사용자의 질문에 대해 친절하게 답변해줘.
  
  [흐름도]
  {details}
  """
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "system",
              "content": system_prompt
          },
          {
              "role": "user",
              "content": query,
          }
      ],
      model="gpt-4o-mini",
  )
  return chat_completion.choices[0].message.content

def multimodal_cot_answer(image_url, query):
  image_details = generate_image_details(image_url)
  answer = text_based_answer(image_details, query)
  return answer


query_list = ["흐름도에서 업무생성에 대해 설명해주세요", "replan은 언제 발생하나요?"]
image_url = "https://langchain-ai.github.io/langgraph/tutorials/plan-and-execute/img/plan-and-execute.png"

for q in query_list:
  print(f"Q: {q}")
  print(f"A: {multimodal_cot_answer(image_url,  q)}")
  print("\n===============================\n")

Q: 흐름도에서 업무생성에 대해 설명해주세요
A: 흐름도에서 "업무 생성(Generate Tasks)" 단계는 사용자의 요청에 따라 수행해야 할 구체적인 작업들을 정의하는 단계입니다. 이 단계는 사용자가 요청한 내용에 따라 어떤 작업이 필요할지를 결정하게 됩니다.

이 과정에서는 요청의 내용을 분석하고, 해결하기 위해 필요한 다양한 작업들을 구상하게 됩니다. 예를 들어, 사용자가 특정 정보를 요청했을 때, 그 정보를 수집하거나, 필요한 데이터로 분석할 여러 개의 작업을 생성할 수 있습니다.

업무 생성 단계에서 생성된 작업들은 "작업 목록(Task List)"으로 이어지며, 이후 단계에서 실제로 수행되게 됩니다. 이 단계는 효율적으로 요청을 처리하기 위해 매우 중요한 부분이며, 명확하고 구체적인 작업 목록을 만드는 것이 성공적인 결과를 위해 필수적입니다.


Q: replan은 언제 발생하나요?
A: 재계획(Replan)은 다음과 같은 경우에 발생합니다:

1. **사용자에게 결과를 응답하는 경우 (5a)**: 작업이 완료된 후, 사용자에게 결과를 전달합니다. 이 과정에서 사용자가 추가적인 요청이나 필요를 제기할 수 있습니다.

2. **더 많은 작업이 필요할 때 (5b)**: 작업 실행 후, 상태 업데이트 및 결과에 따라 추가 작업이 필요하다고 판단되거나, 사용자 요청에 따라 더 많은 작업을 계획해야 할 때 재계획이 이루어집니다.

즉, 재계획은 작업 실행 이후에 발생하며, 사용자의 요구나 작업 결과에 따라 필요한 추가 작업을 계획하는 단계입니다.


